In [1]:
import os
import sys
import ipywidgets as widgets
import pyvo
from IPython.display import Image, display, clear_output
import pandas as pd
import glue_jupyter as gj
from astropy.io.votable import from_table, writeto
import base64
from IPython.core.display import HTML
from pycookiecheat import chrome_cookies
import requests
### set path and import the query builder
sys.path.insert(0, os.path.abspath('./'))       
from core import QueryBuilder


# display glue jupyter plot
#
# @param: df: dataframe
# @return: None
def graph(df):
    global out_data
    with graph_out:
        clear_output()
        app = gj.jglue()
        out_data = app.load_data('output_file.csv')
        columns = df.columns
        scatter_viewer = app.scatter2d(x= columns[0],
                                       y= columns[1],
                                       data=out_data,
                                       show=True)
        update_button = widgets.Button(
            description="Update links",
            style=widgets.ButtonStyle(button_color='#C8F7FD'))

        update_button.on_click(update_subset)
        display(scatter_viewer, update_button, subset_out)


# This function listens on an on_click event, when the button is
# clicked it will regenerate all subset links, and display them.
#
# @param: button widget 
# @return: None
def update_subset(b):
    global out_data
    with subset_out:
        clear_output()
        for id in range(0,len(out_data.subsets)):
            link_name = f"Download subset {id + 1}"
            subset_df = out_data.subsets[id].data.to_dataframe()
            b64 = base64.b64encode(subset_df.to_string().encode())
            payload = b64.decode()
            html = f"""<a download="{link_name}" href="data:text;
            base64, {payload}" target="_blank">{link_name}</a>"""
            display(HTML(html))


# This function listens on an on_click event. when the button is
# clicked, it triggers the search function in the query builder
# class to get data from the data base
# 
# @param: button widget 
# @return: None
def Search_button_result(b):
    with table_out:
        clear_output()
        b.disabled = True
        result_table = query_builder.search_query()
        df = result_table.to_table().to_pandas()
        # add the download button here #
        # the button allows user to download d_f to csv file#
        b64 = base64.b64encode(df.to_string().encode()) 
        payload = b64.decode()
        html = f"""<a download="testfile" href="data:text;
        base64,{payload}" target="_blank">Download File</a>"""
        display(HTML(html))
        display(df)
        df.to_csv('output_file.csv', index=False)
        graph(df)
        b.disabled = False


# This function listens on an on_click event. when the button is
# clicked, the function will log in to cadc system with username
# and pasword, and pass the result cookie into the query builder
# class.
#
# @param: button widget
# @return: None
def login_button_clicked(b):
    global username, password, query_builder
    if b.description=='Login':
        try:
            if username.value!='' and password.value!='':
                data = {
                    'username': username.value,
                    'password': password.value}
                headers = {
                    'Content-Type':"""
                    application/x-www-form-urlencoded""",
                    'Accept':'text/plain'}
                url = 'https://ws-cadc.canfar.net/ac/login'
                response = requests.post(url,
                                         data=data,
                                         headers=headers)
                response.raise_for_status()
                cookie = '\"' + response.text + '\"'
                # assign the result cookie to class variable
                # cookie.
                query_builder.cookie = cookie 
                username.layout.visibility = 'hidden' 
                password.layout.visibility = 'hidden' 
                b.description = 'Logout'
        except Exception:
            username.value = ''
            password.value = ''
    # if the button is a log out button clear the cookie
    elif b.description=='Logout':
        query_builder.cookie = ''
        username.layout.visibility = 'visible' 
        password.layout.visibility = 'visible'
        b.description = 'Login'
        username.value = ''
        password.value = ''


############################################
# create all the output widgets.
query_out_out = widgets.Output(
    layout=widgets.Layout(width='100%'))
query_out = widgets.Output(
    layout=widgets.Layout(width='100%'))
query_out.layout.border = "2px solid green"
table_out = widgets.Output(
    layout=widgets.Layout(width='100%'))
graph_out = widgets.Output(
    layout=widgets.Layout(width='100%'))
subset_out = widgets.Output(
    layout=widgets.Layout(width='100%'))
query_builder = QueryBuilder()
search_button = widgets.Button(
            description="SEARCH",
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'),
            style=widgets.ButtonStyle(button_color='#C8F7FD'))
search_button.on_click(Search_button_result)
# insert Search button
query_builder.list_test.append(search_button)
username = widgets.Text(description="username")
password = widgets.Text(description="password")
login_button = widgets.Button(
        description="Login",
        style=widgets.ButtonStyle(button_color='#C8F7FD'))
login_button.on_click(login_button_clicked)
# try to get CADC_SSO cookie from google chrome browser.
try:
    url = 'https://ws-cadc.canfar.net/ac/login'
    cookies = chrome_cookies(url)
    requests.get(url, cookies=cookies)
    # assign cookie value to the class variable cookie
    query_builder.cookie = cookies['CADC_SSO']
    username.layout.visibility = 'hidden' 
    password.layout.visibility = 'hidden'
    login_button.description = 'Logout'
except Exception:
    pass
# assign output data
with query_out:
    query_builder.Start_query()
with query_out_out:
    display(widgets.HBox([username, password, login_button],
                         layout={'right':'-180px'}))
    display(query_out)
# create a tab widget
tab = widgets.Tab()
tab_contents = ['Query', 'Result', 'Plot']
children = [query_out_out, table_out, graph_out]
tab.children = children
for i in range(len(children)):
    tab.set_title(i, tab_contents[i])
# display the tab widget
display(tab)

# TO Do: chrome_cookies does not work with docker, need to fix it.